# Домашнее задание 4. Quora Question Pairs

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
df_train = pd.read_csv('../input/train.csv', low_memory=False)
df_train.set_index('id', inplace=True)
df_train.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def texts_vectorization(vectorizer, texts_1, texts_2):
    vectorizer.fit(pd.concat([texts_1.fillna(''), texts_2.fillna('')]))
    vect_1 = vectorizer.transform(texts_1.fillna(''))
    vect_2 = vectorizer.transform(texts_2.fillna(''))
    return vect_1, vect_2

vect = CountVectorizer(analyzer='char', ngram_range=(1, 1), min_df=0.001, binary=True)

We vectorize the training sample by counting enough frequently occurring 1-grams, 2-grams, 3-grams, 4-grams and 5-grams. The value of tf will reflect only the presence of N-grams in the question under consideration but not their amount. Here we are having to use 1-grams only because of low performance and established resource limits within the kaggle kernels.

In [ ]:
bag_question1, bag_question2 = texts_vectorization(vect, df_train.question1, df_train.question2)
bag_question1.shape, bag_question2.shape

In [ ]:
X = bag_question1 != bag_question2
X.shape

We shall divide the initial sample into a training sample and a control sample to prevent re-training.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, df_train.is_duplicate, test_size=0.1)

To predict probabilities we will use a multi-layer perceptron classifier with two inner layers of 1000 and 50 neurons, respectively. In order for monitoring the quality of the model on the control subsample, we will limit the training of the model to one iteration, additionally permitting reuse of the solution of the previous call to fit as initialization. Here we are having to set the layers sizes to 100 and 5 respectively.

In [ ]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(hidden_layer_sizes=(100, 5), activation='logistic', max_iter=1, warm_start=True, verbose=True)

By choosing a threshold for a priori score, we must take into account that the maximum allowable value is limited by the parameters of the model and the number of N-grams used.

In [ ]:
from sklearn.metrics import log_loss

score = np.inf
while score > 0.58:
    clf.fit(X_train, y_train)
    score = log_loss(y_test, clf.predict_proba(X_test))
    print('score = {}'.format(score))

In [ ]:
df_test = pd.read_csv('../input/test.csv', low_memory=False)
df_test.set_index('test_id', inplace=True)
df_test.head()

In [ ]:
bag_out_question1 = vect.transform(df_test.question1.fillna(''))
bag_out_question2 = vect.transform(df_test.question2.fillna(''))
bag_out_question1.shape, bag_out_question2.shape

In [ ]:
X_out = bag_out_question1 != bag_out_question2
X_out.shape

Let's predict the probabilities of identity questions in pairs of the test sample.

In [ ]:
y_out_proba = clf.predict_proba(X_out)
y_out_proba.shape

In [ ]:
df_submission = df_test.drop(df_test.columns, axis='columns')
df_submission['is_duplicate'] = y_out_proba[:, 1]
df_submission.to_csv('submission.csv')

Our approach allows to achieve a better score than 0.35 with sufficient amount of computing resources.